In [1]:
import scipy.io as sio
import tensorflow as tf
import numpy as np
from google.colab import data_table
import pandas as pd

### 下載資料並進行整理

In [2]:
!wget https://github.com/toonnyy8/mhmc/raw/master/MultimediaProcessing/Data.mat

--2020-09-03 07:48:13--  https://github.com/toonnyy8/mhmc/raw/master/MultimediaProcessing/Data.mat
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/toonnyy8/mhmc/master/MultimediaProcessing/Data.mat [following]
--2020-09-03 07:48:13--  https://raw.githubusercontent.com/toonnyy8/mhmc/master/MultimediaProcessing/Data.mat
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 152437 (149K) [application/octet-stream]
Saving to: ‘Data.mat’

Data.mat            100%[===================>] 148.86K  --.-KB/s    in 0.02s   

2020-09-03 07:48:13 (9.17 MB/s) - ‘Data.mat’ saved [152437/152437]



In [3]:
data = sio.loadmat('Data.mat')

In [4]:
def to_np_array(data):
  nparr = []

  for _datas in data:
    _arr = []
    for _data in _datas:
      _arr.append(_data)
    nparr.append(_arr)

  return np.array(nparr)

In [5]:
channel1 = to_np_array(data['c1']).transpose((1,0,2,3))
channel2 = to_np_array(data['c2']).transpose((1,0,2,3))
e = 0.1
label = to_np_array(data['lable']).transpose((1,0,2,3))
label = np.maximum(label-e,0)
label = np.maximum(label,e/(label.shape[3]-1))

### 使用最大值進行判別

In [65]:
channel_1_argmax_accuracy = []
channel_1_argmax_confusion_matrix = [[0 for j in range(5)] for i in range(5)]
for idx in range(5):
  x_test = channel1[idx*3:(idx+1)*3].reshape((-1,channel1.shape[3]))
  y_test = label[idx*3:(idx+1)*3].reshape((-1,label.shape[3]))
  predict = tf.argmax(x_test,axis=1)
  ground_truth = tf.argmax(y_test,axis=1)
  for j in range(len(predict)):
    channel_1_argmax_confusion_matrix[predict[j]][ground_truth[j]] += 1
  channel_1_argmax_accuracy.append(
    tf.reduce_sum(
      tf.cast(
        tf.equal(
          predict,
          ground_truth
        ),
        'float32'
      )
    )/450
  )
display(pd.DataFrame(channel_1_argmax_confusion_matrix))

,0,1,2,3,4
0,294,12,1,8,11
1,125,330,24,16,32
2,5,97,321,53,13
3,7,4,90,303,28
4,19,7,14,70,366


In [64]:
channel_2_argmax_accuracy = []
channel_2_argmax_confusion_matrix = [[0 for j in range(5)] for i in range(5)]
for idx in range(5):
  x_test = channel2[idx*3:(idx+1)*3].reshape((-1,channel2.shape[3]))
  y_test = label[idx*3:(idx+1)*3].reshape((-1,label.shape[3]))
  predict = tf.argmax(x_test,axis=1)
  ground_truth = tf.argmax(y_test,axis=1)
  for j in range(len(predict)):
    channel_2_argmax_confusion_matrix[predict[j]][ground_truth[j]] += 1
  channel_2_argmax_accuracy.append(
    tf.reduce_sum(
      tf.cast(
        tf.equal(
          predict,
          ground_truth
        ),
        'float32'
      )
    )/450
  )
display(pd.DataFrame(channel_2_argmax_confusion_matrix))

,0,1,2,3,4
0,360,15,15,15,60
1,30,345,30,0,120
2,15,0,375,15,75
3,15,15,0,405,45
4,30,75,30,15,150


In [63]:
channel_sum_argmax_accuracy = []
channel_sum_argmax_confusion_matrix = [[0 for j in range(5)] for i in range(5)]
for idx in range(5):
  x_test = (channel1[idx*3:(idx+1)*3]+channel2[idx*3:(idx+1)*3]).reshape((-1,channel2.shape[3]))
  y_test = label[idx*3:(idx+1)*3].reshape((-1,label.shape[3]))
  predict = tf.argmax(x_test,axis=1)
  ground_truth = tf.argmax(y_test,axis=1)
  for j in range(len(predict)):
    channel_sum_argmax_confusion_matrix[predict[j]][ground_truth[j]] += 1
  channel_sum_argmax_accuracy.append(
    tf.reduce_sum(
      tf.cast(
        tf.equal(
          predict,
          ground_truth
        ),
        'float32'
      )
    )/450
  )
display(pd.DataFrame(channel_sum_argmax_confusion_matrix))

,0,1,2,3,4
0,396,2,1,5,7
1,32,415,8,4,37
2,3,19,411,7,25
3,10,2,19,423,25
4,9,12,11,11,356


In [28]:
argmax_folds = []
for idx in range(5):
  argmax_folds.append({
    'channel 1':round(float(channel_1_argmax_accuracy[idx])*100000)/100000,
    'channel 2':round(float(channel_2_argmax_accuracy[idx])*100000)/100000,
    'channel sum':round(float(channel_sum_argmax_accuracy[idx])*100000)/100000
  })
argmax_folds.append({
  'channel 1':round(float(tf.reduce_mean(channel_1_argmax_accuracy))*100000)/100000,
  'channel 2':round(float(tf.reduce_mean(channel_2_argmax_accuracy))*100000)/100000,
  'channel sum':round(float(tf.reduce_mean(channel_sum_argmax_accuracy))*100000)/100000,
})
argmax_df = pd.DataFrame({
  'fold 0':argmax_folds[0],
  'fold 1':argmax_folds[1],
  'fold 2':argmax_folds[2],
  'fold 3':argmax_folds[3],
  'fold 4':argmax_folds[4],
  'average':argmax_folds[5],
})

data_table.DataTable(argmax_df)

,fold 0,fold 1,fold 2,fold 3,fold 4,average
channel 1,0.72000,0.73111,0.67778,0.72667,0.73111,0.71733
channel 2,0.72667,0.72667,0.72667,0.72667,0.72667,0.72667
channel sum,0.85556,0.90222,0.90000,0.89556,0.89333,0.88933


### 建立模型

In [2]:
def build_model(output):
  model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(16,tf.keras.activations.swish),
    tf.keras.layers.Dense(16,tf.keras.activations.swish),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(16,tf.keras.activations.swish),
    tf.keras.layers.Dense(16,tf.keras.activations.swish),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(16,tf.keras.activations.swish),
    tf.keras.layers.Dense(output,tf.keras.activations.softmax),
  ])
  model.compile(tf.optimizers.Adam(),tf.losses.categorical_crossentropy,['accuracy'])
  return model

#### 模型結構

In [3]:
channel_model = build_model(5)
channel_model.build((None,5))
channel_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                96        
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
batch_normalization (BatchNo (None, 16)                64        
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_3 (Dense)              (None, 16)                272       
_________________________________________________________________
batch_normalization_1 (Batch (None, 16)                64        
_________________________________________________________________
dense_4 (Dense)              (None, 16)                2

In [3]:
connect_model = build_model(5)
connect_model.build((None,10))
connect_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                176       
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
batch_normalization (BatchNo (None, 16)                64        
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_3 (Dense)              (None, 16)                272       
_________________________________________________________________
batch_normalization_1 (Batch (None, 16)                64        
_________________________________________________________________
dense_4 (Dense)              (None, 16)                2

### Model 1
只使用 channel 1

#### 訓練

In [11]:
models_1 = [] 
for i in range(5):
  model = build_model(5)
  x_train = np.concatenate([channel1[:(i)*3],channel1[(i+1)*3:]]).reshape((-1,channel1.shape[3]))
  y_train = np.concatenate([label[:(i)*3],label[(i+1)*3:]]).reshape((-1,label.shape[3]))
  model.fit(x_train, y_train, epochs=500)
  models_1.append(model)

串流輸出內容已截斷至最後 5000 行。
57/57 [==============================] - 0s 5ms/step - loss: 1.5279 - accuracy: 0.4244
Epoch 2/500
57/57 [==============================] - 0s 5ms/step - loss: 1.1269 - accuracy: 0.6828
Epoch 3/500
57/57 [==============================] - 0s 5ms/step - loss: 0.9731 - accuracy: 0.7572
Epoch 4/500
57/57 [==============================] - 0s 5ms/step - loss: 0.9232 - accuracy: 0.7717
Epoch 5/500
57/57 [==============================] - 0s 5ms/step - loss: 0.8974 - accuracy: 0.7822
Epoch 6/500
57/57 [==============================] - 0s 5ms/step - loss: 0.8928 - accuracy: 0.7817
Epoch 7/500
57/57 [==============================] - 0s 5ms/step - loss: 0.8629 - accuracy: 0.8011
Epoch 8/500
57/57 [==============================] - 0s 5ms/step - loss: 0.8562 - accuracy: 0.7950
Epoch 9/500
57/57 [==============================] - 0s 5ms/step - loss: 0.8579 - accuracy: 0.7972
Epoch 10/500
57/57 [==============================] - 0s 5ms/step - loss: 0.8607 - accuracy: 0.7922


#### 測試

In [32]:
models_1_accuracy = []
models_1_confusion_matrix = [[0 for j in range(5)] for i in range(5)]
for idx, model in enumerate(models_1):
  x_test = channel1[idx*3:(idx+1)*3].reshape((-1,channel1.shape[3]))
  y_test = label[idx*3:(idx+1)*3].reshape((-1,label.shape[3]))
  predict = tf.argmax(model.predict(x_test),axis=1)
  ground_truth = tf.argmax(y_test,axis=1)
  for j in range(len(predict)):
    models_1_confusion_matrix[predict[j]][ground_truth[j]] += 1
  models_1_accuracy.append(
    tf.reduce_sum(
      tf.cast(
        tf.equal(
          predict,
          ground_truth
        ),
        'float32'
      )
    )/450
  )
  print('fold {} accuracy'.format(idx))
  print(models_1_accuracy[idx])
  print('------')

print('average accuracy')
print(tf.reduce_mean(models_1_accuracy))

fold 0 accuracy
tf.Tensor(0.8088889, shape=(), dtype=float32)
------
fold 1 accuracy
tf.Tensor(0.8244445, shape=(), dtype=float32)
------
fold 2 accuracy
tf.Tensor(0.72, shape=(), dtype=float32)
------
fold 3 accuracy
tf.Tensor(0.7911111, shape=(), dtype=float32)
------
fold 4 accuracy
tf.Tensor(0.79333335, shape=(), dtype=float32)
------
average accuracy
tf.Tensor(0.7875556, shape=(), dtype=float32)


In [25]:
pd.DataFrame(models_1_confusion_matrix)

,0,1,2,3,4
0,396,35,2,23,18
1,8,347,31,4,16
2,7,50,378,81,25
3,13,3,22,288,28
4,26,15,17,54,363


### Model 2
只使用 channel 2

#### 訓練

In [13]:
models_2 = []
for i in range(5):
  model = build_model(5)
  x_train = np.concatenate([channel2[:(i)*3],channel2[(i+1)*3:]]).reshape((-1,channel2.shape[3]))
  y_train = np.concatenate([label[:(i)*3],label[(i+1)*3:]]).reshape((-1,label.shape[3]))
  model.fit(x_train, y_train, epochs=500)
  models_2.append(model)

串流輸出內容已截斷至最後 5000 行。
57/57 [==============================] - 0s 5ms/step - loss: 1.5234 - accuracy: 0.3344
Epoch 2/500
57/57 [==============================] - 0s 5ms/step - loss: 1.3327 - accuracy: 0.4811
Epoch 3/500
57/57 [==============================] - 0s 5ms/step - loss: 1.2196 - accuracy: 0.5783
Epoch 4/500
57/57 [==============================] - 0s 5ms/step - loss: 1.1355 - accuracy: 0.6378
Epoch 5/500
57/57 [==============================] - 0s 5ms/step - loss: 1.0650 - accuracy: 0.6667
Epoch 6/500
57/57 [==============================] - 0s 5ms/step - loss: 1.0163 - accuracy: 0.7006
Epoch 7/500
57/57 [==============================] - 0s 5ms/step - loss: 1.0023 - accuracy: 0.7111
Epoch 8/500
57/57 [==============================] - 0s 5ms/step - loss: 0.9685 - accuracy: 0.7244
Epoch 9/500
57/57 [==============================] - 0s 5ms/step - loss: 0.9446 - accuracy: 0.7328
Epoch 10/500
57/57 [==============================] - 0s 5ms/step - loss: 0.9444 - accuracy: 0.7294


#### 測試

In [41]:
models_2_accuracy = []
models_2_confusion_matrix = [[0 for j in range(5)] for i in range(5)]
for idx, model in enumerate(models_2):
  x_test = channel2[idx*3:(idx+1)*3].reshape((-1,channel2.shape[3]))
  y_test = label[idx*3:(idx+1)*3].reshape((-1,label.shape[3]))
  predict = tf.argmax(model.predict(x_test),axis=1)
  ground_truth = tf.argmax(y_test,axis=1)
  for j in range(len(predict)):
    models_2_confusion_matrix[predict[j]][ground_truth[j]] += 1
  models_2_accuracy.append(
    tf.reduce_sum(
      tf.cast(
        tf.equal(
          predict,
          ground_truth
        ),
        'float32'
      )
    )/450
  )
  print('fold {} accuracy'.format(idx))
  print(models_2_accuracy[idx])
  print('------')

print('average accuracy')
print(tf.reduce_mean(models_2_accuracy))

fold 0 accuracy
tf.Tensor(0.92, shape=(), dtype=float32)
------
fold 1 accuracy
tf.Tensor(0.96666664, shape=(), dtype=float32)
------
fold 2 accuracy
tf.Tensor(0.9866667, shape=(), dtype=float32)
------
fold 3 accuracy
tf.Tensor(0.93333334, shape=(), dtype=float32)
------
fold 4 accuracy
tf.Tensor(0.99333334, shape=(), dtype=float32)
------
average accuracy
tf.Tensor(0.96000004, shape=(), dtype=float32)


In [33]:
pd.DataFrame(models_2_confusion_matrix)

,0,1,2,3,4
0,423,0,0,6,9
1,0,435,6,0,0
2,0,0,441,0,0
3,9,0,0,420,0
4,18,15,3,24,441


### Model 3
將 channel 1 與 channel 2 一起使用

#### 訓練

In [15]:
models_3 = [] 
for i in range(5):
  connect_channel = np.concatenate([channel1,channel2],axis=3)
  model = build_model(5)
  x_train = np.concatenate([connect_channel[:(i)*3],connect_channel[(i+1)*3:]]).reshape((-1,connect_channel.shape[3]))
  y_train = np.concatenate([label[:(i)*3],label[(i+1)*3:]]).reshape((-1,label.shape[3]))
  model.fit(x_train, y_train, epochs=500)
  models_3.append(model)

串流輸出內容已截斷至最後 5000 行。
57/57 [==============================] - 0s 5ms/step - loss: 1.3917 - accuracy: 0.4556
Epoch 2/500
57/57 [==============================] - 0s 5ms/step - loss: 1.0562 - accuracy: 0.6961
Epoch 3/500
57/57 [==============================] - 0s 5ms/step - loss: 0.8829 - accuracy: 0.8022
Epoch 4/500
57/57 [==============================] - 0s 5ms/step - loss: 0.7885 - accuracy: 0.8556
Epoch 5/500
57/57 [==============================] - 0s 5ms/step - loss: 0.7485 - accuracy: 0.8839
Epoch 6/500
57/57 [==============================] - 0s 5ms/step - loss: 0.7355 - accuracy: 0.8739
Epoch 7/500
57/57 [==============================] - 0s 5ms/step - loss: 0.7234 - accuracy: 0.8844
Epoch 8/500
57/57 [==============================] - 0s 5ms/step - loss: 0.7217 - accuracy: 0.8778
Epoch 9/500
57/57 [==============================] - 0s 5ms/step - loss: 0.7120 - accuracy: 0.8883
Epoch 10/500
57/57 [==============================] - 0s 5ms/step - loss: 0.6932 - accuracy: 0.9039


#### 測試

In [40]:
models_3_accuracy = []
models_3_confusion_matrix = [[0 for j in range(5)] for i in range(5)]
for idx, model in enumerate(models_3):
  connect_data = np.concatenate([channel1,channel2],axis=3)
  x_test = connect_data[idx*3:(idx+1)*3].reshape((-1,connect_data.shape[3]))
  y_test = label[idx*3:(idx+1)*3].reshape((-1,label.shape[3]))
  predict = tf.argmax(model.predict(x_test),axis=1)
  ground_truth = tf.argmax(y_test,axis=1)
  for j in range(len(predict)):
    models_3_confusion_matrix[predict[j]][ground_truth[j]] += 1
  models_3_accuracy.append(
    tf.reduce_sum(
      tf.cast(
        tf.equal(
          predict,
          ground_truth
        ),
        'float32'
      )
    )/450
  )
  print('fold {} accuracy'.format(idx))
  print(models_3_accuracy[idx])
  print('------')

print('average accuracy')
print(tf.reduce_mean(models_3_accuracy))

fold 0 accuracy
tf.Tensor(0.9488889, shape=(), dtype=float32)
------
fold 1 accuracy
tf.Tensor(0.9577778, shape=(), dtype=float32)
------
fold 2 accuracy
tf.Tensor(0.8888889, shape=(), dtype=float32)
------
fold 3 accuracy
tf.Tensor(0.9355556, shape=(), dtype=float32)
------
fold 4 accuracy
tf.Tensor(0.96, shape=(), dtype=float32)
------
average accuracy
tf.Tensor(0.9382222, shape=(), dtype=float32)


In [37]:
pd.DataFrame(models_3_confusion_matrix)

,0,1,2,3,4
0,436,7,0,6,9
1,10,414,11,2,20
2,0,17,428,11,8
3,3,4,4,429,9
4,1,8,7,2,404


### Model 4
將 channel 1 與 channel 2 相加

#### 訓練

In [17]:
models_4 = [] 
for i in range(5):
  sum_channel = channel1 + channel2
  model = build_model(5)
  x_train = np.concatenate([sum_channel[:(i)*3],sum_channel[(i+1)*3:]]).reshape((-1,sum_channel.shape[3]))
  y_train = np.concatenate([label[:(i)*3],label[(i+1)*3:]]).reshape((-1,label.shape[3]))
  model.fit(x_train, y_train, epochs=500)
  models_4.append(model)

串流輸出內容已截斷至最後 5000 行。
57/57 [==============================] - 0s 5ms/step - loss: 1.3924 - accuracy: 0.4394
Epoch 2/500
57/57 [==============================] - 0s 5ms/step - loss: 1.1456 - accuracy: 0.6683
Epoch 3/500
57/57 [==============================] - 0s 5ms/step - loss: 0.9895 - accuracy: 0.7839
Epoch 4/500
57/57 [==============================] - 0s 5ms/step - loss: 0.8848 - accuracy: 0.8267
Epoch 5/500
57/57 [==============================] - 0s 4ms/step - loss: 0.8030 - accuracy: 0.8617
Epoch 6/500
57/57 [==============================] - 0s 5ms/step - loss: 0.7729 - accuracy: 0.8650
Epoch 7/500
57/57 [==============================] - 0s 5ms/step - loss: 0.7569 - accuracy: 0.8783
Epoch 8/500
57/57 [==============================] - 0s 5ms/step - loss: 0.7394 - accuracy: 0.8861
Epoch 9/500
57/57 [==============================] - 0s 5ms/step - loss: 0.7263 - accuracy: 0.8761
Epoch 10/500
57/57 [==============================] - 0s 5ms/step - loss: 0.7266 - accuracy: 0.8894


#### 測試

In [38]:
models_4_accuracy = []
models_4_confusion_matrix = [[0 for j in range(5)] for i in range(5)]
for idx, model in enumerate(models_4):
  sum_channel = channel1 + channel2
  x_test = sum_channel[idx*3:(idx+1)*3].reshape((-1,sum_channel.shape[3]))
  y_test = label[idx*3:(idx+1)*3].reshape((-1,label.shape[3]))
  predict = tf.argmax(model.predict(x_test),axis=1)
  ground_truth = tf.argmax(y_test,axis=1)
  for j in range(len(predict)):
    models_4_confusion_matrix[predict[j]][ground_truth[j]] += 1
  models_4_accuracy.append(
    tf.reduce_sum(
      tf.cast(
        tf.equal(
          predict,
          ground_truth
        ),
        'float32'
      )
    )/450
  )
  print('fold {} accuracy'.format(idx))
  print(models_4_accuracy[idx])
  print('------')

print('average accuracy')
print(tf.reduce_mean(models_4_accuracy))

fold 0 accuracy
tf.Tensor(0.9088889, shape=(), dtype=float32)
------
fold 1 accuracy
tf.Tensor(0.94222224, shape=(), dtype=float32)
------
fold 2 accuracy
tf.Tensor(0.9355556, shape=(), dtype=float32)
------
fold 3 accuracy
tf.Tensor(0.9533333, shape=(), dtype=float32)
------
fold 4 accuracy
tf.Tensor(0.9311111, shape=(), dtype=float32)
------
average accuracy
tf.Tensor(0.9342222, shape=(), dtype=float32)


In [42]:
pd.DataFrame(models_4_confusion_matrix)

,0,1,2,3,4
0,430,5,2,7,11
1,1,425,2,1,17
2,3,0,425,4,14
3,11,2,9,432,18
4,5,18,12,6,390


### 模型表格

In [26]:
model_folds = []
for idx in range(5):
  model_folds.append({
    'channel 1':round(float(models_1_accuracy[idx])*100000)/100000,
    'channel 2':round(float(models_2_accuracy[idx])*100000)/100000,
    'connect channel':round(float(models_3_accuracy[idx])*100000)/100000,
    'channel sum':round(float(models_4_accuracy[idx])*100000)/100000
  })
model_folds.append({
  'channel 1':round(float(tf.reduce_mean(models_1_accuracy))*100000)/100000,
  'channel 2':round(float(tf.reduce_mean(models_2_accuracy))*100000)/100000,
  'connect channel':round(float(tf.reduce_mean(models_3_accuracy))*100000)/100000,
  'channel sum':round(float(tf.reduce_mean(models_4_accuracy))*100000)/100000,
})
model_df = pd.DataFrame({
  'fold 0':model_folds[0],
  'fold 1':model_folds[1],
  'fold 2':model_folds[2],
  'fold 3':model_folds[3],
  'fold 4':model_folds[4],
  'average':model_folds[5],
})

### 總結
從 表 1 來看可發現，只靠最大值而不使用類神經網路來判斷結果時,channel 1 與 channel 2 的效果相差不多，但若將兩者相加後再作判別，便可大幅提升準確率。  

依據 表 3 的實驗結果來看，在 channel 1 中應該有較多不乾淨的資料存在，使模型的決策受到影響，與 channel 2 **串接**一起使用反而**降低**平均正確率。  

如果是將 channel 1 與 channel 2 **相加**後在由模型判別，與串接相比，兩者在不同的 fold 上有各自的優勢，因此相加與串接的平均準確率相差不大。


In [58]:
print('表 1、由 channel 的最大值判別')
display(data_table.DataTable(argmax_df))

表 1、由 channel 的最大值判別


,fold 0,fold 1,fold 2,fold 3,fold 4,average
channel 1,0.72000,0.73111,0.67778,0.72667,0.73111,0.71733
channel 2,0.72667,0.72667,0.72667,0.72667,0.72667,0.72667
channel sum,0.85556,0.90222,0.90000,0.89556,0.89333,0.88933


In [62]:
print('表 2、最大值判別的混淆矩陣(predict\ground truth)\n')
print('channel 1')
display(pd.DataFrame(channel_1_argmax_confusion_matrix))
print('\nchannel 2')
display(pd.DataFrame(channel_2_argmax_confusion_matrix))
print('\nchannel sum')
display(pd.DataFrame(channel_sum_argmax_confusion_matrix))

表 2、最大值判別的混淆矩陣(predict\ground truth)

channel 1


,0,1,2,3,4
0,294,12,1,8,11
1,125,330,24,16,32
2,5,97,321,53,13
3,7,4,90,303,28
4,19,7,14,70,366



channel 2


,0,1,2,3,4
0,360,15,15,15,60
1,30,345,30,0,120
2,15,0,375,15,75
3,15,15,0,405,45
4,30,75,30,15,150



channel sum


,0,1,2,3,4
0,396,2,1,5,7
1,32,415,8,4,37
2,3,19,411,7,25
3,10,2,19,423,25
4,9,12,11,11,356


In [60]:
print('表 3、模型正確率')
display(data_table.DataTable(model_df))

表 3、模型正確率


,fold 0,fold 1,fold 2,fold 3,fold 4,average
channel 1,0.80889,0.82444,0.72000,0.79111,0.79333,0.78756
channel 2,0.92000,0.96667,0.98667,0.93333,0.99333,0.96000
connect channel,0.94889,0.95778,0.88889,0.93556,0.96000,0.93822
channel sum,0.90889,0.94222,0.93556,0.95333,0.93111,0.93422


In [61]:
print('表 4、模型的混淆矩陣(predict\ground truth)\n')
print('Model 1')
display(pd.DataFrame(models_1_confusion_matrix))
print('\nModel 2')
display(pd.DataFrame(models_2_confusion_matrix))
print('\nModel 3')
display(pd.DataFrame(models_3_confusion_matrix))
print('\nModel 4')
display(pd.DataFrame(models_4_confusion_matrix))

表 4、模型的混淆矩陣(predict\ground truth)

Model 1


,0,1,2,3,4
0,396,35,2,23,18
1,8,347,31,4,16
2,7,50,378,81,25
3,13,3,22,288,28
4,26,15,17,54,363



Model 2


,0,1,2,3,4
0,423,0,0,6,9
1,0,435,6,0,0
2,0,0,441,0,0
3,9,0,0,420,0
4,18,15,3,24,441



Model 3


,0,1,2,3,4
0,436,7,0,6,9
1,10,414,11,2,20
2,0,17,428,11,8
3,3,4,4,429,9
4,1,8,7,2,404



Model 4


,0,1,2,3,4
0,430,5,2,7,11
1,1,425,2,1,17
2,3,0,425,4,14
3,11,2,9,432,18
4,5,18,12,6,390
